In [26]:
import pandas as pd
import matplotlib.pyplot as plt 

In [27]:
skinnerData = pd.read_csv("skinner.csv")
hellyData = pd.read_csv("hellebuyck.csv")
skinnerData

,Player,Date,Game_ID,Season,Team,Opponent,Is_Home,Position,TOI,TOI%,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAx
0,Stuart Skinner,2023-10-11,2023020009,20232024,EDM,VAN,0,G,32.50,54.17,3.96,17.26,19.40,1.36,77.03,79.57,93.01,-13.44,-2.61
1,Stuart Skinner,2023-10-14,2023020029,20232024,EDM,VAN,1,G,57.63,96.06,4.19,16.48,24.94,2.41,74.58,83.21,90.35,-7.14,-1.78
2,Stuart Skinner,2023-10-21,2023020077,20232024,EDM,WPG,1,G,63.73,99.77,2.98,26.40,35.33,2.70,88.71,91.56,92.36,-0.80,-0.28
3,Stuart Skinner,2023-10-26,2023020110,20232024,EDM,NYR,1,G,60.00,100.00,3.22,35.18,50.02,2.88,90.85,93.56,94.25,-0.69,-0.34
4,Stuart Skinner,2023-10-29,2023020129,20232024,EDM,CGY,1,G,59.75,99.58,2.08,25.38,44.84,3.07,91.80,95.36,93.16,2.20,0.99
5,Stuart Skinner,2023-11-02,2023020152,20232024,EDM,DAL,1,G,57.83,96.39,4.38,29.80,32.27,2.89,85.32,86.44,91.04,-4.60,-1.49
6,Stuart Skinner,2023-11-06,2023020179,20232024,EDM,VAN,0,G,60.00,100.00,5.85,37.76,49.48,4.41,84.50,88.17,91.09,-2.91,-1.44
7,Stuart Skinner,2023-11-09,2023020203,20232024,EDM,S.J,0,G,58.12,96.86,2.88,18.05,30.02,1.80,84.06,90.41,94.01,-3.60,-1.08
8,Stuart Skinner,2023-11-11,2023020220,20232024,EDM,SEA,0,G,60.00,100.00,0.87,15.85,22.04,1.31,94.53,96.07,94.06,2.01,0.44
9,Stuart Skinner,2023-11-13,2023020227,20232024,EDM,NYI,1,G,60.00,100.00,1.06,33.78,48.61,3.07,96.86,97.82,93.68,4.14,2.01


In [28]:
skinnerStartData = skinnerData.iloc[range(13)]
skinnerStartData.head()


,Player,Date,Game_ID,Season,Team,Opponent,Is_Home,Position,TOI,TOI%,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAx
0,Stuart Skinner,2023-10-11,2023020009,20232024,EDM,VAN,0,G,32.50,54.17,3.96,17.26,19.40,1.36,77.03,79.57,93.01,-13.44,-2.61
1,Stuart Skinner,2023-10-14,2023020029,20232024,EDM,VAN,1,G,57.63,96.06,4.19,16.48,24.94,2.41,74.58,83.21,90.35,-7.14,-1.78
2,Stuart Skinner,2023-10-21,2023020077,20232024,EDM,WPG,1,G,63.73,99.77,2.98,26.40,35.33,2.70,88.71,91.56,92.36,-0.80,-0.28
3,Stuart Skinner,2023-10-26,2023020110,20232024,EDM,NYR,1,G,60.00,100.00,3.22,35.18,50.02,2.88,90.85,93.56,94.25,-0.69,-0.34
4,Stuart Skinner,2023-10-29,2023020129,20232024,EDM,CGY,1,G,59.75,99.58,2.08,25.38,44.84,3.07,91.80,95.36,93.16,2.20,0.99


In [29]:
skinnerEndData = skinnerData.iloc[range(13, 33)]
skinnerEndData.head()

,Player,Date,Game_ID,Season,Team,Opponent,Is_Home,Position,TOI,TOI%,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAx
13,Stuart Skinner,2023-11-24,2023020298,20232024,EDM,WSH,0,G,60.00,100.00,0.00,22.02,29.13,1.98,100.00,100.00,93.21,6.79,1.98
14,Stuart Skinner,2023-11-26,2023020320,20232024,EDM,ANA,1,G,58.65,97.75,2.12,22.44,32.31,1.29,90.55,93.43,96.01,-2.58,-0.83
15,Stuart Skinner,2023-11-28,2023020335,20232024,EDM,VGK,1,G,65.00,100.00,4.05,26.94,33.87,2.49,84.97,88.05,92.64,-4.59,-1.55
16,Stuart Skinner,2023-11-30,2023020349,20232024,EDM,WPG,0,G,59.90,99.83,0.93,26.17,35.22,3.03,96.43,97.35,91.40,5.95,2.09
17,Stuart Skinner,2023-12-06,2023020391,20232024,EDM,CAR,1,G,60.00,100.00,1.01,36.96,49.85,3.51,97.26,97.97,92.95,5.02,2.50


In [30]:
skinnerStartData.loc[""] 

KeyError: ''